# krackhardt

> This modula contains a Python implementation of the Krackhardt hierarchy score

In [1]:
#| default_exp krackhardt

In [2]:
#| hide
from nbdev.showdoc import *
import os, pandas as pd, numpy as np
import networkx

import rpy2, rpy2.situation
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr

## Krackhardt OG approach

From his OG paper:

The graph hierarchy condition states that in a digraph D, for each pair of points where one (Pi) can reach another (Pj), the second (Pj) can't reach the first (Pi). 
For example, in a formal organization chart a high lvl employee can reach through the chain of command her subordinate's subordinate. If the formal organization if working "properly", this lower lvl employee can't simultaneously reach the high lvl employee.
To measure the degree of hierarchy of digraph D, a new digraph Dr must be created. Dr is defined as the reachability digraph of D. Each point in D exists in Dr; moreover, the line (Pi,Pj) exists in Dr if and only if Pi can reach Pj in D. If D is graph hierarchic, then Dr will have no symmetric lines in it (i.e. if the line (Pi,Pj) exists in Dr then the line (Pj,Pi) does not).

The degree of hierarchy then is defined as:

    Graph Hierarchy = 1 - [V/MaxV]
    Where:
        V = Number of unordered pairs of points in Dr that are symmetrically linked
        MaxV = Number of unordered pairs of points in Dr where Pi is linked to Pj or viceversa.

**Definitions**:

- Hierarchy degree: This a network-wide metric that determines how "hierarchical" a graph is.
From https://doi.org/10.1186/s13059-015-0624-2:
    The degree of hierarchy for a given for a given network is not well-defined concept. Ispolatov et al. [27] introduced the idea of dominant direction by minimizing the number of feedback links. While it is a proxy of hierarchical structure to a certain extent, the method does not provide a rigorous statistical confidence. Here, we define a metric to quantify the degree of hierarchy for a given hierarchical network, and then propose a new method called hierarchical score maximization (HSM) to infer the hierarchy of a directed network

- Hierarchy metric (at node level): From sna package in R.
    Hierarchy measures quantify the extent of asymmetry in a structure; the greater the extent of asym-
    metry, the more hierarchical the structure is said to be. (This should not be confused with how
    centralized the structure is, i.e., the extent to which centralities of vertex positions are highly con-
    centrated.)
They discuss and offer 2 different approaches:

    - reciprocity: This setting returns one minus the dyadic reciprocity for each input graph (see
    grecip)
    - krackhardt: This setting returns the Krackhardt hierarchy score for each input graph. The
    Krackhardt hierarchy is defined as the fraction of non-null dyads in the reachability graph
    which are asymmetric. Thus, when no directed paths are reciprocated (e.g., in an in/outtree),
    Krackhardt hierarchy is equal to 1; when all such paths are reciprocated, by contrast (e.g., in
    a cycle or clique), the measure falls to 0.
        - Hierarchy is one of four measures (connectedness, efficiency, hierarchy, and lubness)
        suggested by Krackhardt for summarizing hierarchical structures. Each corresponds to one of
        four axioms which are necessary and sufficient for the structure in question to be an outtree;
        thus, the measures will be equal to 1 for a given graph iff that graph is an outtree. Deviations
        from unity can be interpreted in terms of failure to satisfy one or more of the outtree conditions,
        information which may be useful in classifying its structural properties.
        Note that hierarchy is inherently density-constrained: as densities climb above 0.5, the proportion
        of mutual dyads must (by the pigeonhole principle) increase rapidly, thereby reducing possibili-
        ties for asymmetry. Thus, the interpretation of hierarchy scores should take density into account,
        particularly if density is artifactual (e.g., due to a particular dichotomization procedure)
    
- Flow Hierarchy Score: Described in XXXX (paper). Implementation from the networkX package.

In [32]:
#| export
def compute_hierarchy(G, metric="pykrack"):
    "Compute one of 3 hierarchy scores"

    #Ensure Graph is DirectedGraph
    if not G.is_directed():
        raise Exception
    #Ensure Graph is of DiGraph() format
    G = networkx.DiGraph(G)

    if metric == "pykrack": #Python implementation
        #Compute transitive closure of graph to get the reachability graph
            #[contains an edge (i,j) if there is a path from i to j in the original graph]
        acyclic = 0
        try:
            networkx.find_cycle(G)
        except:
            print("Acyclic graph")
            acyclic = 1
        if acyclic == 1:
            Gr = networkx.transitive_closure_dag(G)
        else:
            Gr = networkx.transitive_closure(G, reflexive=None)
        asymmetric_dyads = 0
        non_null_dyads = 0
        n = len(Gr.nodes())
        #Count the number of non-null asymmetric dyads
        for i in Gr.nodes():
            for j in Gr.nodes():
                if Gr.has_edge(i,j) or Gr.has_edge(j,i): #Non-null dyad
                    non_null_dyads+=1
                    if  Gr.has_edge(i,j) != Gr.has_edge(j,i): #Asymmetric!
                        asymmetric_dyads+=1
        #Raise exception if graph has no edges!
        if non_null_dyads == 0:
            raise Exception
        score = asymmetric_dyads / non_null_dyads
    
    elif metric == "rsnakrack": #R implementation from the sna package
        base = importr("base")
        sna = importr("sna")
        score = sna.hierarchy(networkx.to_numpy_array(G), measure="krackhardt")
    
    elif metric == "hierarchy_flow": #Networkx's hierarchy flow implementation
        score = networkx.flow_hierarchy(G)

    else: # metric argument broken
        raise Exception



    return score


In [31]:
#| export
G = networkx.DiGraph([(1, 2), (2, 3), (1, 5), (2, 4), (4, 6), (5, 6), (3, 1)])

In [8]:
G = networkx.DiGraph([(1, 2), (2, 3), (1, 5), (2, 4), (4, 6), (5, 6)])

In [27]:
G = networkx.DiGraph([(1, 2), (4, 6)])

In [47]:
#Graph from OmniPath
#using the following link: https://omnipathdb.org/interactions?genesymbols=yes&fields=type&license=academic&directed=0
#Gets us all interactions, no matter type nor undirected/vs directed
omnidata = pd.read_csv("https://omnipathdb.org/interactions?genesymbols=yes&fields=type&license=academic&directed=1", sep="\t")

G = networkx.from_pandas_edgelist(omnidata, source="source_genesymbol", target="target_genesymbol",edge_attr="type", create_using=networkx.DiGraph)
G

In [49]:
compute_hierarchy(G, metric="pykrack")

0.7813765887749392

In [48]:
compute_hierarchy(G, metric="hierarchy_flow")

0.33794232464425134

In [46]:
pd.read_csv("https://omnipathdb.org/interactions?genesymbols=yes&fields=type&license=academic&directed=1", sep="\t")

,source,target,source_genesymbol,target_genesymbol,is_directed,is_stimulation,is_inhibition,consensus_direction,consensus_stimulation,consensus_inhibition,type
0,P0DP25,P48995,CALM3,TRPC1,1,0,1,1,0,1,post_translational
1,P0DP23,P48995,CALM1,TRPC1,1,0,1,1,0,1,post_translational
2,P0DP24,P48995,CALM2,TRPC1,1,0,1,1,0,1,post_translational
3,Q03135,P48995,CAV1,TRPC1,1,1,0,1,1,0,post_translational
4,P14416,P48995,DRD2,TRPC1,1,1,0,1,1,0,post_translational
...,...,...,...,...,...,...,...,...,...,...,...
40009,P08581,O14964,MET,HGS,1,1,0,1,1,0,post_translational
40010,P08581,Q6VN20,MET,RANBP10,1,0,0,0,0,0,post_translational
40011,P06239,P16070,LCK,CD44,1,0,0,0,0,0,post_translational
40012,P04626,Q07889,ERBB2,SOS1,1,1,0,1,1,0,post_translational


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()